# 5 Minute Tutorial Series

## Fixing Your Mistakes by Updating and Overwriting Data

### Setup the Environment

In [18]:
import os
import sys
import tempfile
import pandas as pd
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

### Connect to the GIS

In [2]:
gis = GIS(profile='your_online_profile')

## Load the Sample Data

- in this tutorial the data used will be downloaded from the ArcGIS Open Data Site


In [4]:
df = pd.read_csv("https://opendata.arcgis.com/datasets/dc3289eab3d2400ea49c154863312434_8.csv")
df.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
0,-77.045290,38.907248,20400331,2020-03-11T16:05:16.000Z,EVENING,OTHERS,THEFT/OTHER,2000 - 2007 BLOCK OF N STREET NW,396072.0,137710.0,...,010700 2,10700,Precinct 4,38.907240,-77.045287,GOLDEN TRIANGLE,2020-02-24T15:55:00.000Z,2020-03-11T15:40:00.000Z,420637740,20400331-01
1,-77.018024,38.911868,20400332,2020-03-11T19:46:08.000Z,EVENING,OTHERS,THEFT/OTHER,400 - 499 BLOCK OF WARNER STREET NW,398437.0,138222.0,...,004801 1,4801,Precinct 18,38.911860,-77.018022,NaN,2020-03-11T11:49:00.000Z,2020-03-11T17:25:00.000Z,420637741,20400332-01
2,-77.027267,38.930720,20400333,2020-02-25T20:52:56.000Z,EVENING,OTHERS,THEFT/OTHER,1000 - 1099 BLOCK OF LAMONT STREET NW,397636.0,140315.0,...,003100 1,3100,Precinct 38,38.930713,-77.027264,NaN,2020-02-25T17:00:00.000Z,2020-02-25T18:00:00.000Z,420637742,20400333-01
3,-77.008219,38.875556,20400335,2020-03-11T11:03:48.000Z,DAY,OTHERS,THEFT/OTHER,1200 - 1299 BLOCK OF VAN STREET SE,399287.0,134191.0,...,007200 2,7200,Precinct 131,38.875548,-77.008217,CAPITOL RIVERFRONT,2020-03-07T07:00:00.000Z,2020-03-09T06:00:00.000Z,420637743,20400335-01
4,-76.994371,38.826622,20400338,2020-03-13T13:59:21.000Z,DAY,OTHERS,THEFT F/AUTO,734 - 855 BLOCK OF SOUTHERN AVENUE SE,400489.0,128759.0,...,009811 2,9811,Precinct 125,38.826615,-76.994369,NaN,2020-02-13T12:40:00.000Z,2020-03-13T12:40:00.000Z,420637744,20400338-01


In [8]:
split_spot = len(df)//2
half_df = df[:split_spot].copy()

## Add the Sample Data to GIS 

In [9]:
half_df.to_csv("./myuploaddata.csv")

In [10]:
item = gis.content.add({
    'title' : 'overwrite_source',
    'type' : 'CSV',
    'tags' : "demo, data"
},
data="./myuploaddata.csv")
item

<Item title:"overwrite_source" type:CSV owner:andrew57>

In [15]:
crime_locations = item.publish()
crime_locations

<Item title:"overwrite_source" type:Feature Layer Collection owner:andrew57>

### OH NO WE FORGOT HALF THE DATA

<img src="./img/shame.jpg"/>

- how do we fix this?

### Enter Overwriting Data

- new data can be uploaded to the service and republished.  
- the `overwrite` parameter needs to be set to `True`
- the csv file needs to the same exact file name and same column names 
    + if you do not have this, you will fail 

In [20]:
import os
with tempfile.TemporaryDirectory() as tmp:
    fp = os.path.join(tmp, "myuploaddata.csv") # same name CHECK
    df.to_csv(fp)
    flc = FeatureLayerCollection.fromitem(crime_locations)
    flc.manager.overwrite(fp)

**What Happened Here?**

1. Write the data to a temporary folder
2. Access the item's `Feature Service`
3. Access the manager (owners and admins only)
4. Call `overwrite` and pass in the new dataset

In [25]:
new_count = crime_locations.layers[0].query(return_count_only=True) 
old_count = len(half_df)
print(new_count, old_count, new_count > old_count)

2216 1108 True


**Shame No More!**